**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Dropout

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Student__: Mathis Petrovich

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer__ : ```epsilon``` is essential because it allows the agent to explore the actions in order to learn about them and improve its chances of getting future reward.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=2 
epochs_test=5 

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer__:

-  ```board``` correspond to the reward associed with each place of the grid (bonus/malus)
- ```position``` correspond to the collision map. 0 for nothing, 1 for the player and -1 for the border

In [24]:
# make a folder for vids
!mkdir videos

mkdir: cannot create directory ‘videos’: File exists


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Pick randomly an action
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1, epochs+1):
        state = env.reset()
        
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='videos/random')
HTML(display_videos('videos/random1.mp4'))

Win/lose count 7.0/12.0. Average score (-2.5)
Win/lose count 7.5/17.0. Average score (-4.833333333333333)
Win/lose count 15.5/18.0. Average score (-4.25)
Win/lose count 13.0/18.0. Average score (-4.4)
Win/lose count 5.5/12.0. Average score (-4.75)
Final score: -5.7


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

__Proof__:

\begin{align*}
Q^\pi(s,a) &= E_{p^{\pi}}\left[\sum_{0 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \\
Q^\pi(s,a) &= r(s,a) + E_{p^{\pi}}\left[\sum_{1 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \\
Q^\pi(s,a) &= r(s,a) + \gamma E_{p^{\pi}}\left[\sum_{0 \leq t}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a\right] \\
Q^\pi(s,a) &= r(s,a) + \gamma \sum_{s', \ a'} p(s'|s,a) \pi(a'|s') E_{p^{\pi}}\left[\sum_{0 \leq t}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a, s_1=s', a_1=a'\right] \\
Q^\pi(s,a) &= r(s,a) + \gamma \sum_{s', \ a'} p(s'|s,a) \pi(a'|s') E_{p^{\pi}}\left[\sum_{0 \leq t}\gamma^{t}r(s_{t+1},a_{t+1})|s_1=s', a_1=a'\right]  \ \text{By Markov property}\\
Q^\pi(s,a) &= r(s,a) + \gamma \sum_{s', \ a'} p(s'|s,a) \pi(a'|s') Q^{\pi}(s',a') \\
Q^{\pi}(s,a)& =E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim p^{\pi^*}(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

__Proof__:
\begin{align*}
Q^{*}(s,a) &= Q^{\pi^{*}}(s,a) = E_{(s',a')\sim p^{\pi^{*}}(.|s,a)}[r(s,a)+\gamma Q^{\pi^{*}}(s',a')] \\
 &= r(s,a) + \gamma \sum_{s', \ a'} p(s'|s,a) \pi(a'|s') Q^{\pi^{*}}(s',a') \\
 &= r(s,a) + \gamma \sum_{s'} p(s'|s,a) \sum_{a'} \pi(a'|s') Q^{\pi^{*}}(s',a')
\end{align*}

Because $Q^*(s,a)=\max_{\pi}Q^\pi(s,a)$, the action given by the policy $\pi^{*}$ have to maximize $Q^{*}$ so

\begin{align*}
 \sum_{a'} \pi(a'|s') Q^{\pi^{*}}(s',a') = \max_{a'}Q^{*}(s',a')
\end{align*}
 
And thus:
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim p^{\pi^*}(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim p^{\pi^*}(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

__Proof__:
Here $Q(s,a,\theta)$ is a vector, this is why we use a norm.

To be able to optain our $Q^{*}$, this iterative method works:

\begin{equation*}
Q(s,a) \leftarrow E_{s'\sim p^{\pi^*}(.|s,a)}[r(s,a)+\gamma\max_{a'}Q(s',a')].
\end{equation*}

To be able to see the convergence and improve parameters $\gamma$ we can look at the square of the norm of the difference between 2 iterations:

\begin{align*}
\mathcal{L}(\theta) &= \left\Vert Q_{new}(s,a) - Q_{old}(s,a) \right\Vert^{2} \\
\mathcal{L}(\theta) &= \left\Vert E_{s' \sim \pi^*(.|s,a)} \left[ r+\gamma\max_{a'}Q(s',a',\theta) \right]  - Q(s,a,\theta) \right\Vert^{2} \\
\mathcal{L}(\theta) &= E_{s' \sim \pi^*(.|s,a)} \left\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta) \right\Vert^{2}
\end{align*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory.pop(0)
            self.memory.append(m)
            
    def random_access(self):
        # Pick an index randomly then take it from the memory
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        bs = s.reshape(1, 5, 5, self.n_state)
        return np.argmax(self.model.predict(bs))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
                
        # Learn from past
        for i in range(self.batch_size):
            # Take a state/reward/action memory
            s, n_s, a, r, game_over = self.memory.random_access()
            
            # For other action, don't change anything
            target_q[i] = self.model.predict(s.reshape(1, 5, 5, self.n_state))
        
            if not game_over_:
                target_q[i][a] = r + self.discount * np.max(self.model.predict(n_s.reshape(1, 5, 5, self.n_state)))
            else:
                target_q[i][a] = r
                
            input_states[i] = s
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(Flatten(input_shape = (5,5,self.n_state)))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(4, activation="linear"))
        
        model.compile("adam", "mse")
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [11]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100

In [44]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(size, epsilon = 0.1, memory_size=200, batch_size = 32)
train(agent, env, epochs_train, prefix='videos/fc_train')

Epoch 001/100 | Loss 0.0441 | Win/lose count 7.0/7.0 (0.0)
Epoch 002/100 | Loss 1.8059 | Win/lose count 7.0/1.0 (6.0)
Epoch 003/100 | Loss 2.1212 | Win/lose count 5.0/4.0 (1.0)
Epoch 004/100 | Loss 1.6266 | Win/lose count 8.0/6.0 (2.0)
Epoch 005/100 | Loss 1.6176 | Win/lose count 6.0/2.0 (4.0)
Epoch 006/100 | Loss 1.8201 | Win/lose count 0.5/0 (0.5)
Epoch 007/100 | Loss 1.5152 | Win/lose count 6.5/5.0 (1.5)
Epoch 008/100 | Loss 1.7920 | Win/lose count 3.5/5.0 (-1.5)
Epoch 009/100 | Loss 1.6824 | Win/lose count 1.0/1.0 (0.0)
Epoch 010/100 | Loss 1.9825 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/100 | Loss 1.8434 | Win/lose count 3.0/3.0 (0.0)
Epoch 012/100 | Loss 1.4637 | Win/lose count 3.0/2.0 (1.0)
Epoch 013/100 | Loss 1.6874 | Win/lose count 7.5/3.0 (4.5)
Epoch 014/100 | Loss 1.4101 | Win/lose count 2.0/1.0 (1.0)
Epoch 015/100 | Loss 1.4571 | Win/lose count 8.0/7.0 (1.0)
Epoch 016/100 | Loss 1.6487 | Win/lose count 4.5/3.0 (1.5)
Epoch 017/100 | Loss 1.7571 | Win/lose count 6.5/1.0 (5.

In [45]:
HTML(display_videos('videos/fc_train100.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [12]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()

        model.add(Conv2D(32, (2, 2), activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(16, (2, 2), activation='relu'))
        model.add(Dropout(0.1))
        model.add(Flatten())
        model.add(Dense(20, activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(4, activation='linear'))
        
        model.compile("adam", "mse")
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [13]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200

In [48]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, epsilon = 0.1, memory_size=200, batch_size = 32)
train(agent,env,epochs_train,prefix='videos/cnn_train')

Epoch 001/200 | Loss 0.0476 | Win/lose count 6.0/4.0 (2.0)
Epoch 002/200 | Loss 0.0192 | Win/lose count 3.5/2.0 (1.5)
Epoch 003/200 | Loss 0.0067 | Win/lose count 1.0/0 (1.0)
Epoch 004/200 | Loss 0.0108 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/200 | Loss 0.0519 | Win/lose count 4.0/3.0 (1.0)
Epoch 006/200 | Loss 0.3897 | Win/lose count 11.5/3.0 (8.5)
Epoch 007/200 | Loss 0.5403 | Win/lose count 11.0/6.0 (5.0)
Epoch 008/200 | Loss 0.6157 | Win/lose count 11.0/4.0 (7.0)
Epoch 009/200 | Loss 0.5346 | Win/lose count 8.5/1.0 (7.5)
Epoch 010/200 | Loss 0.4254 | Win/lose count 11.0/5.0 (6.0)
Epoch 011/200 | Loss 0.4669 | Win/lose count 10.0/4.0 (6.0)
Epoch 012/200 | Loss 0.2970 | Win/lose count 5.0/5.0 (0.0)
Epoch 013/200 | Loss 0.1707 | Win/lose count 7.0/5.0 (2.0)
Epoch 014/200 | Loss 0.1881 | Win/lose count 6.0/4.0 (2.0)
Epoch 015/200 | Loss 0.2153 | Win/lose count 9.0/5.0 (4.0)
Epoch 016/200 | Loss 1.0348 | Win/lose count 19.5/5.0 (14.5)
Epoch 017/200 | Loss 0.4115 | Win/lose count 5.5/4.

Epoch 137/200 | Loss 1.4991 | Win/lose count 18.5/4.0 (14.5)
Epoch 138/200 | Loss 1.5595 | Win/lose count 22.5/4.0 (18.5)
Epoch 139/200 | Loss 1.3955 | Win/lose count 7.0/2.0 (5.0)
Epoch 140/200 | Loss 1.3671 | Win/lose count 13.0/2.0 (11.0)
Epoch 141/200 | Loss 1.4931 | Win/lose count 23.5/5.0 (18.5)
Epoch 142/200 | Loss 1.6074 | Win/lose count 20.5/3.0 (17.5)
Epoch 143/200 | Loss 1.5620 | Win/lose count 14.0/2.0 (12.0)
Epoch 144/200 | Loss 1.5278 | Win/lose count 26.5/4.0 (22.5)
Epoch 145/200 | Loss 1.5995 | Win/lose count 13.0/4.0 (9.0)
Epoch 146/200 | Loss 1.5948 | Win/lose count 20.5/3.0 (17.5)
Epoch 147/200 | Loss 1.4863 | Win/lose count 15.5/4.0 (11.5)
Epoch 148/200 | Loss 1.4870 | Win/lose count 25.0/9.0 (16.0)
Epoch 149/200 | Loss 1.3341 | Win/lose count 10.0/3.0 (7.0)
Epoch 150/200 | Loss 1.5195 | Win/lose count 21.5/6.0 (15.5)
Epoch 151/200 | Loss 1.5011 | Win/lose count 17.0/8.0 (9.0)
Epoch 152/200 | Loss 1.3806 | Win/lose count 13.0/3.0 (10.0)
Epoch 153/200 | Loss 1.6220 |

In [60]:
HTML(display_videos('videos/cnn_train200.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [14]:
epochs_test=10

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, epsilon = 0.1, memory_size=200, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, epsilon = 0.1, memory_size=200, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='videos/cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='videos/fc_test')

Test of the CNN
Win/lose count 22.5/1.0. Average score (10.75)
Win/lose count 18.5/3.0. Average score (12.333333333333334)
Win/lose count 13.0/2.0. Average score (12.0)
Win/lose count 22.5/2.0. Average score (13.7)
Win/lose count 16.5/1.0. Average score (14.0)
Win/lose count 27.5/0. Average score (15.928571428571429)
Win/lose count 18.5/0. Average score (16.25)
Win/lose count 19.5/4.0. Average score (16.166666666666668)
Win/lose count 22.5/2.0. Average score (16.6)
Win/lose count 19.0/3.0. Average score (16.545454545454547)
Final score: 18.2
Test of the FC
Win/lose count 15.5/1.0. Average score (7.25)
Win/lose count 2.0/0. Average score (5.5)
Win/lose count 6.0/0. Average score (5.625)
Win/lose count 1.5/0. Average score (4.8)
Win/lose count 2.0/0. Average score (4.333333333333333)
Win/lose count 15.5/0. Average score (5.928571428571429)
Win/lose count 8.0/0. Average score (6.1875)
Win/lose count 9.0/0. Average score (6.5)
Win/lose count 5.0/0. Average score (6.35)
Win/lose count 10.5/

In [35]:
HTML(display_videos('videos/cnn_test9.mp4'))

In [17]:
HTML(display_videos('videos/fc_test10.mp4'))

**Observation** : The CNN works way better thatn the fully connected one.

**Answer** : I notice that the agent didn't explore well the map. The agent get stuck somewhere instead of searching more bonus.

__Test with different temperatures__

In [18]:
agent_cnn = DQN_CNN(size, epsilon = 0.1, memory_size=200, batch_size = 32, n_state = 2)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')
agent_fc = DQN_FC(size, epsilon = 0.1, memory_size=200, batch_size = 32, n_state = 2)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

In [23]:
def test_temp(temp):
    env = Environment(grid_size=size, max_time=T,temperature=temp)

    print('Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='videos/cnn_test_temp' + str(temp))
    print('Test of the FC')
    test(agent_fc,env,epochs_test,prefix='videos/fc_test_temp' + str(temp))

In [20]:
test_temp(0.2)

Test of the CNN
Win/lose count 4.5/0. Average score (2.25)
Win/lose count 3.0/0. Average score (2.5)
Win/lose count 1.0/0. Average score (2.125)
Win/lose count 12.0/1.0. Average score (3.9)
Win/lose count 18.5/1.0. Average score (6.166666666666667)
Win/lose count 8.0/0. Average score (6.428571428571429)
Win/lose count 10.0/5.0. Average score (6.25)
Win/lose count 12.5/3.0. Average score (6.611111111111111)
Win/lose count 12.5/1.0. Average score (7.1)
Win/lose count 16.0/5.0. Average score (7.454545454545454)
Final score: 8.2
Test of the FC
Win/lose count 9.0/0. Average score (4.5)
Win/lose count 2.0/1.0. Average score (3.3333333333333335)
Win/lose count 3.5/0. Average score (3.375)
Win/lose count 11.5/1.0. Average score (4.8)
Win/lose count 3.5/0. Average score (4.583333333333333)
Win/lose count 5.0/0. Average score (4.642857142857143)
Win/lose count 5.5/0. Average score (4.75)
Win/lose count 3.0/0. Average score (4.555555555555555)
Win/lose count 1.0/0. Average score (4.2)
Win/lose co

In [21]:
test_temp(0.5)

Test of the CNN
Win/lose count 32.5/3.0. Average score (14.75)
Win/lose count 36.5/5.0. Average score (20.333333333333332)
Win/lose count 31.0/1.0. Average score (22.75)
Win/lose count 16.5/3.0. Average score (20.9)
Win/lose count 15.0/0. Average score (19.916666666666668)
Win/lose count 14.0/2.0. Average score (18.785714285714285)
Win/lose count 18.0/3.0. Average score (18.3125)
Win/lose count 17.5/5.0. Average score (17.666666666666668)
Win/lose count 41.5/3.0. Average score (19.75)
Win/lose count 26.0/2.0. Average score (20.136363636363637)
Final score: 22.15
Test of the FC
Win/lose count 6.0/1.0. Average score (2.5)
Win/lose count 5.5/0. Average score (3.5)
Win/lose count 6.5/0. Average score (4.25)
Win/lose count 7.5/0. Average score (4.9)
Win/lose count 20.5/1.0. Average score (7.333333333333333)
Win/lose count 4.0/0. Average score (6.857142857142857)
Win/lose count 13.5/1.0. Average score (7.5625)
Win/lose count 9.0/0. Average score (7.722222222222222)
Win/lose count 4.5/1.0. Av

In [22]:
test_temp(0.8)

Test of the CNN
Win/lose count 43.0/1.0. Average score (21.0)
Win/lose count 65.5/2.0. Average score (35.166666666666664)
Win/lose count 56.0/1.0. Average score (40.125)
Win/lose count 67.5/0. Average score (45.6)
Win/lose count 62.5/2.0. Average score (48.083333333333336)
Win/lose count 32.5/0. Average score (45.857142857142854)
Win/lose count 63.0/1.0. Average score (47.875)
Win/lose count 66.5/0. Average score (49.94444444444444)
Win/lose count 59.5/1.0. Average score (50.8)
Win/lose count 59.0/0. Average score (51.54545454545455)
Final score: 56.7
Test of the FC
Win/lose count 9.0/1.0. Average score (4.0)
Win/lose count 43.0/1.0. Average score (16.666666666666668)
Win/lose count 18.5/0. Average score (17.125)
Win/lose count 29.5/0. Average score (19.6)
Win/lose count 29.5/0. Average score (21.25)
Win/lose count 9.0/0. Average score (19.5)
Win/lose count 37.0/1.0. Average score (21.5625)
Win/lose count 32.5/0. Average score (22.77777777777778)
Win/lose count 27.5/0. Average score (2

__Observation__:

We can notice that when we increase the temperature, the agent tend to explore more. It is indeed lead by the bonus.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [27]:
def train_explore(agent,env,epoch,prefix='', epsilon_begin=1, epsilon_end=0):
    # Number of won games
    score = 0
    loss = 0
   
    for e in range(1, epoch+1):
        # Adapt epsilon
        epsilon = epsilon_end*(e-1)/epoch + epsilon_begin * (epoch-e+1)/epoch
        agent.set_epsilon(epsilon)

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, tot_reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, tot_reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.malus_position = np.zeros((grid_size,grid_size))
   
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        explore_reward = - self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] = 0.1
        
        reward = self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        tot_reward = explore_reward + reward
        return state, reward, tot_reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [33]:
epochs_train = 500

In [34]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, memory_size=200, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='videos/cnn_train_explore', epsilon_begin=1, epsilon_end=0)

Epoch 001/500 | Loss 0.0431 | Win/lose count 9.0/18.0 (-9.0)
Epoch 002/500 | Loss 0.7502 | Win/lose count 12.0/10.0 (2.0)
Epoch 003/500 | Loss 0.4013 | Win/lose count 9.0/14.0 (-5.0)
Epoch 004/500 | Loss 0.5481 | Win/lose count 8.0/15.0 (-7.0)
Epoch 005/500 | Loss 1.0469 | Win/lose count 10.0/10.0 (0.0)
Epoch 006/500 | Loss 0.9636 | Win/lose count 12.0/20.0 (-8.0)
Epoch 007/500 | Loss 0.8564 | Win/lose count 11.5/15.0 (-3.5)
Epoch 008/500 | Loss 0.9383 | Win/lose count 15.0/15.0 (0.0)
Epoch 009/500 | Loss 1.1905 | Win/lose count 16.0/14.0 (2.0)
Epoch 010/500 | Loss 0.9270 | Win/lose count 9.0/21.0 (-12.0)
Epoch 011/500 | Loss 1.1207 | Win/lose count 10.0/17.0 (-7.0)
Epoch 012/500 | Loss 0.9155 | Win/lose count 11.5/9.0 (2.5)
Epoch 013/500 | Loss 1.4014 | Win/lose count 11.0/11.0 (0.0)
Epoch 014/500 | Loss 1.2733 | Win/lose count 13.0/13.0 (0.0)
Epoch 015/500 | Loss 1.2834 | Win/lose count 11.0/16.0 (-5.0)
Epoch 016/500 | Loss 1.3259 | Win/lose count 8.5/12.0 (-3.5)
Epoch 017/500 | Loss

Epoch 135/500 | Loss 1.7115 | Win/lose count 13.5/8.0 (5.5)
Epoch 136/500 | Loss 1.6715 | Win/lose count 12.0/13.0 (-1.0)
Epoch 137/500 | Loss 1.8487 | Win/lose count 19.0/15.0 (4.0)
Epoch 138/500 | Loss 1.7634 | Win/lose count 19.0/9.0 (10.0)
Epoch 139/500 | Loss 1.7845 | Win/lose count 14.5/16.0 (-1.5)
Epoch 140/500 | Loss 1.7471 | Win/lose count 15.0/13.0 (2.0)
Epoch 141/500 | Loss 1.7263 | Win/lose count 21.0/10.0 (11.0)
Epoch 142/500 | Loss 1.6744 | Win/lose count 16.5/12.0 (4.5)
Epoch 143/500 | Loss 1.7732 | Win/lose count 22.0/15.0 (7.0)
Epoch 144/500 | Loss 1.7997 | Win/lose count 21.0/11.0 (10.0)
Epoch 145/500 | Loss 1.7727 | Win/lose count 14.0/12.0 (2.0)
Epoch 146/500 | Loss 1.6138 | Win/lose count 7.0/7.0 (0.0)
Epoch 147/500 | Loss 1.6166 | Win/lose count 17.5/11.0 (6.5)
Epoch 148/500 | Loss 1.6589 | Win/lose count 16.0/16.0 (0.0)
Epoch 149/500 | Loss 1.4754 | Win/lose count 7.5/4.0 (3.5)
Epoch 150/500 | Loss 1.8630 | Win/lose count 22.0/9.0 (13.0)
Epoch 151/500 | Loss 1.77

Epoch 270/500 | Loss 1.7171 | Win/lose count 21.5/13.0 (8.5)
Epoch 271/500 | Loss 1.4149 | Win/lose count 9.0/8.0 (1.0)
Epoch 272/500 | Loss 1.5446 | Win/lose count 17.0/12.0 (5.0)
Epoch 273/500 | Loss 1.6268 | Win/lose count 20.0/10.0 (10.0)
Epoch 274/500 | Loss 1.5291 | Win/lose count 19.5/9.0 (10.5)
Epoch 275/500 | Loss 1.5965 | Win/lose count 22.0/6.0 (16.0)
Epoch 276/500 | Loss 1.5588 | Win/lose count 22.5/8.0 (14.5)
Epoch 277/500 | Loss 1.5430 | Win/lose count 24.0/10.0 (14.0)
Epoch 278/500 | Loss 1.3619 | Win/lose count 16.5/8.0 (8.5)
Epoch 279/500 | Loss 1.4970 | Win/lose count 22.0/11.0 (11.0)
Epoch 280/500 | Loss 1.6268 | Win/lose count 24.0/12.0 (12.0)
Epoch 281/500 | Loss 1.4085 | Win/lose count 14.5/13.0 (1.5)
Epoch 282/500 | Loss 1.4990 | Win/lose count 23.5/10.0 (13.5)
Epoch 283/500 | Loss 1.4028 | Win/lose count 19.0/9.0 (10.0)
Epoch 284/500 | Loss 1.3296 | Win/lose count 14.5/9.0 (5.5)
Epoch 285/500 | Loss 1.2356 | Win/lose count 21.0/9.0 (12.0)
Epoch 286/500 | Loss 1.

Epoch 405/500 | Loss 0.5125 | Win/lose count 15.0/5.0 (10.0)
Epoch 406/500 | Loss 0.6828 | Win/lose count 25.0/7.0 (18.0)
Epoch 407/500 | Loss 0.7111 | Win/lose count 18.0/9.0 (9.0)
Epoch 408/500 | Loss 0.9977 | Win/lose count 21.0/12.0 (9.0)
Epoch 409/500 | Loss 0.3862 | Win/lose count 15.0/14.0 (1.0)
Epoch 410/500 | Loss 0.9955 | Win/lose count 20.0/12.0 (8.0)
Epoch 411/500 | Loss 0.7582 | Win/lose count 12.5/5.0 (7.5)
Epoch 412/500 | Loss 1.0086 | Win/lose count 23.0/7.0 (16.0)
Epoch 413/500 | Loss 1.0005 | Win/lose count 22.5/13.0 (9.5)
Epoch 414/500 | Loss 1.3154 | Win/lose count 21.5/3.0 (18.5)
Epoch 415/500 | Loss 1.1813 | Win/lose count 26.5/8.0 (18.5)
Epoch 416/500 | Loss 1.2776 | Win/lose count 22.0/5.0 (17.0)
Epoch 417/500 | Loss 0.9533 | Win/lose count 19.0/2.0 (17.0)
Epoch 418/500 | Loss 0.7175 | Win/lose count 18.5/10.0 (8.5)
Epoch 419/500 | Loss 0.9475 | Win/lose count 15.0/4.0 (11.0)
Epoch 420/500 | Loss 0.4859 | Win/lose count 11.0/3.0 (8.0)
Epoch 421/500 | Loss 0.5278

In [36]:
HTML(display_videos('videos/cnn_train_explore500.mp4'))

In [37]:
def test_explore(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1, epochs+1):
        state = env.reset()
        
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, tot_reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [40]:
epochs_test = 10

# Evaluation
test_explore(agent,env,epochs_test,prefix='videos/cnn_test_explore')

Win/lose count 25.5/2.0. Average score (11.75)
Win/lose count 17.5/5.0. Average score (12.0)
Win/lose count 29.0/1.0. Average score (16.0)
Win/lose count 2.0/0. Average score (13.2)
Win/lose count 18.0/2.0. Average score (13.666666666666666)
Win/lose count 22.5/3.0. Average score (14.5)
Win/lose count 18.0/0. Average score (14.9375)
Win/lose count 24.5/3.0. Average score (15.666666666666666)
Win/lose count 14.5/5.0. Average score (15.05)
Win/lose count 13.5/4.0. Average score (14.545454545454545)
Final score: 16.0


In [59]:
HTML(display_videos('videos/cnn_test_explore9.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***